### Challenges associated with streaming data

#### Ingesting variable volumes
- massive amounts of streaming events, handle spiky/bursting data, high availability and durability
- Cloud Pub/Sub (Ingest)

#### Late data, unordered data
- how to deal with latency, late arriving records, or speculative results
- Data Dataflow (Processing & Imperative Analysis)

#### Real-time insights
- continuous query processing, visualization, analytics, etc.
- Google BigQuery (Durable storage and interactive analysis)

## Module 1 Review

1.) Dataflow offers the following that makes it easy to create resilient streaming pipelines when working with unbounded data
- Ability to flexibly reason about time
- Control messages to ensure correctness

2.) Match the GCP product with its role when designing streaming systems
- Pub / Sub: Global messaging queue
- Dataflow: Controls to handle late-arriving and out-of-order data
- Bigtable: latency in the order of milliseconds when querying against overwhelming volume
- BiqQuery: Query data as it arrives from streaming pipelines

## Lab: Publish Streaming Data into Pub/Sub
#### Objectives:
- Create a Pub/Sub topic and subscription
- Simulate your traffic sensor data into Pub/Sub

#### Task 1: Preparation
- In the Console, on the Navigation menu () click Compute Engine > VM instances.
- Locate the line with the instance called training_vm.
- On the far right, under 'connect', Click on SSH to open a terminal window.
- In this lab you will enter CLI commands on the training_vm.
- The training_vm is installing software in the background. Verify that setup is complete by checking that the following directory exists. If it does not exist, wait a few minutes and try again
- A repository has been downloaded to the VM. Copy the repository to your home directory.

```
ls /training
# copy to home directory
cp -r /training/training-data-analyst/ .
```
- On the training_vm SSH terminal, set the DEVSHELL_PROJECT_ID environment variable and export it so it will be available to other shells.
```
export DEVSHELL_PROJECT_ID=<project-id>
```

#### Task 2: Create Pub/Sub topic and subscription
- On the training_vm SSH terminal, navigate to the directory for this lab.
```
cd ~/training-data-analyst/courses/streaming/publish
gcloud pubsub topics create sandiego
gcloud pubsub topics publish sandiego --message "hello"
gcloud pubsub subscriptions create --topic sandiego mySub1
gcloud pubsub subscriptions pull --auto-ack mySub1
# try again
gcloud pubsub topics publish sandiego --message "hello again"
gcloud pubsub subscriptions pull --auto-ack mySub1
```

- Return to the Console tab. On the Navigation menu () click Pub/Sub > Topics.
- You should see a line with the Topic Name ending in sandiego and the number of Subscriptions set to 1.
- In the training_vm SSH terminal,, cancel your subscription.

```
gcloud pubsub subscriptions delete mySub1
```

#### Task 3: Simulate traffic sensor data into Pub/Sub
- Explore the python script to simulate San Diego traffic sensor data. Do not make any changes to the code.
```
cd ~/training-data-analyst/courses/streaming/publish
nano send_sensor_data.py
```
- Download the traffic simulation dataset.
```
./download_data.sh
sudo apt-get install -y python-pip
sudo pip install -U google-cloud-pubsub
./send_sensor_data.py --speedFactor=60 --project (dollar sign)DEVSHELL_PROJECT_ID
```

- This command simulates sensor data by sending recorded sensor data via Pub/Sub messages. The script extracts the original time of the sensor data and pauses between sending each message to simulate realistic timing of the sensor data. The value speedFactor changes the time between messages proportionally. So a speedFactor of 60 means '60 times faster' than the recorded timing. It will send about an hour of data every 60 seconds.

#### Task 4: Verify that messages are received
- In the Console, on the Navigation menu () click Compute Engine > VM instances.
- Locate the line with the instance called training_vm.
- On the far right, under 'connect', Click on SSH to open a second terminal window.
- Change into the directory you were working in:
```
cd ~/training-data-analyst/courses/streaming/publish
gcloud pubsub subscriptions create --topic sandiego mySub2
gcloud pubsub subscriptions pull --auto-ack mySub2
# cancel subscription
gcloud pubsub subscriptions delete mySub2
exit
```

## End Lab

## Module 2 Review

1.) Which of the following about Cloud Pub/Sub is NOT true?
- Pub/Sub stores your messages indefinitely until you need it

Pub/Sub does:
- Simplify systems by removing the need for every component to speak to every component
- Connect applications and devices through a messaging infrastructure

2.) Cloud Pub/Sub guarantees that messages delivered are in the order they were received
- False
(Pub/Sub takes advantage of timestamping to deliver in the correct order)

3.) Which of the following about Cloud Pub/Sub topics and subscriptions are true?
- 1 or more publishers can write to the same topic
- 1 or more subscribers can request from the same subscription

4.) Which of the following delivery methods is ideal for subscribers needing close to real time performance?
- Push delivery 

## Lab: Streaming Data Pipelines
#### Objectives:
- Launch Dataflow and run a Dataflow job
- Understand how data elements flow through the transformations of a Dataflow pipeline
- Connect Dataflow to Pub/Sub and BigQuery
- Observe and understand how Dataflow autoscaling adjusts compute resources to process input data optimally
- Learn where to find logging information created by Dataflow
- Explore metrics and create alerts and dashboards with Stackdriver Monitoring

#### Task 1: Preparation
- In the Console, on the Navigation menu () click Compute Engine > VM instances.
- Locate the line with the instance called training_vm.
- On the far right, under 'connect', Click on SSH to open a terminal window.
- In this lab you will enter CLI commands on the training_vm.
```
ls /training
cp -r /training/training-data-analyst/ .
source /training/project_env.sh
```

#### Task 2: Create a BigQuery Dataset and Cloud Storage Bucket
- Open the BigQuery web UI. On the Navigation menu () click BigQuery.
- In the left column, beneath the text box, find your project name. To the right of the project name, click the blue arrow. Choose Create new dataset.
- In the ‘Create Dataset' dialog, for Dataset ID, type demos and click OK.
- In the Console, on the Navigation menu () click Storage > Browser.

#### Task 3: Simulate traffic sensor data into Pub/Sub
- In the training_vm SSH terminal, start the sensor simulator. The script reads sample data from a csv file and publishes it to Pub/Sub.
```
/training/sensor_magic.sh
```
- In the Console, on the Navigation menu () click Pub/Sub>Topics
- Examine the line for Topic name for the topic sandiego. Notice that Subscriptions are currently at 0.
```
source /training/project_env.sh
```
#### Task 4: Launch Dataflow Pipeline
- Return to the browser tab for Console. In the top search bar, enter Dataflow API. This will take you to the page, Navigation > APIs & Services > Dashboard > Google Dataflow API. It will either show a status information or it will give you the option to Enable the API.
- If necessary, Enable the API.
- Return to the second training_vm SSH terminal. Change into the directory for this lab.
```
cd ~/training-data-analyst/courses/streaming/process/sandiego
cat run_oncloud.sh
# github source code
# https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/streaming/process/sandiego/run_oncloud.sh
# check out java directory
cd ~/training-data-analyst/courses/streaming/process/sandiego/src/main/java/com/google/cloud/training/dataanalyst/sandiego 
cat AverageSpeeds.java
# github source code
# https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/streaming/process/sandiego/src/main/java/com/google/cloud/training/dataanalyst/sandiego/AverageSpeeds.java
# build a Dataflow streaming pipeline
cd ~/training-data-analyst/courses/streaming/process/sandiego
./run_oncloud.sh $DEVSHELL_PROJECT_ID $BUCKET AverageSpeeds
```

#### Task 5: Explore the pipeline
- Return to the browser tab for Console. On the Navigation menu () click Dataflow and click on your job to monitor progress.
- After the pipeline is running, click on the Navigation menu () click Pub/Sub>Topics.
- Examine the line for Topic name for the topic sandiego. Notice that Subscriptions field is now at 1.
- Return to the Navigation menu () click Dataflow and click on your job.
- Compare the code in the Github browser tab, AverageSpeeds.java and the pipeline graph in the page for your Dataflow job.
- Find the "GetMessages" pipeline step in the graph, and then find the corresponding code in the AverageSpeeds.java file. This is the pipeline step that reads from the Pub/Sub topic. It creates a collection of Strings - which corresponds to Pub/Sub messages that have been read.
- Do you see a subscription created?
- How does the code pull messages from Pub/Sub?
- Find the "Time Window" pipeline step in the graph and in code. In this pipeline step we create a window of a duration specified in the pipeline parameters (sliding window in this case). This window will accumulate the traffic data from the previous step until end of window, and pass it to the next steps for further transforms.
- What is the window interval?
- How often is a new window created?
- Find the "BySensor" and "AvgBySensor" pipeline steps in the graph, and then find the corresponding code snippet in the AverageSpeeds.java file. This "BySensor" does a grouping of all events in the window by sensor id, while "AvgBySensor" will then compute the mean speed for each grouping.
- Find the "ToBQRow" pipeline step in the graph and in code. This step simply creates a "row" with the average computed from previous step together with the lane information.
- In practice, other actions could be taken in the ToBQRow step. For example, it could compare the calculated mean against a predefined threshold and log the results of the comparison in Stackdriver Logging.

- Find the "BigQueryIO.Write" in both the pipeline graph and in the source code. This step writes the row out of the pipeline into a BigQuery table. Because we chose the WriteDisposition.WRITE_APPEND write disposition, new records will be appended to the table.
- Return to the BigQuery web UI tab Or open it from the Navigation menu () click BigQuery. Refresh your browser.
- In the left column, beneath the text box, find your project name and the demos dataset you created. The small blue arrow to the left should now be active and clicking on it will reveal the average_speeds table. 

#### Task 6: Determine throughput rates
- Return to the browser tab for Console. On the Navigation menu () click Dataflow and click on your job to monitor progress (it will have your username in the pipeline name).
- Select the "GetMessages" pipeline node in the graph and look at the step metrics on the right.
- System Lag is an important metric for streaming pipelines. It represents the amount of time data elements are waiting to be processed since they "arrived" in the input of the transformation step.
- Elements Added metric under output collections tells you how many data elements exited this step (for the "Read PubSub Msg" step of the pipeline it also represents the number of Pub/Sub messages read from the topic by the Pub/Sub IO connector).
- Select the "Time Window" node in the graph. Observe how the Elements Added metric under the Input Collections of the "Time -Window" step matches the Elements Added metric under the Output Collections of the previous step "GetMessages".

#### Task 7: Review BigQuery output
- Return to the BigQuery web UI or on the Navigation menu () click BigQuery.
- Use the following query to observe the output from the Dataflow job. Replace <PROJECTID> with your Project ID. It is listed under connection details in Qwiklabs.

```
SELECT * 
FROM [<PROJECTID>:demos.average_speeds] 
ORDER BY timestamp DESC
LIMIT 100
```
- Find the last update to the table by running the following SQL.

```
SELECT
  MAX(timestamp)
FROM
  [<PROJECTID>:demos.average_speeds]
```

- Use the BigQuery Table Decorator to look at results in the last 10 minutes.
```
SELECT
  *
FROM
  [<PROJECTID>:demos.average_speeds@-600000]
ORDER BY
  timestamp DESC
```

#### Task 8: Observe and understand autoscaling
- Return to the browser tab for Console. On the Navigation menu () click Dataflow and click on your pipeline job.
- Examine the Job summary panel on the right, and review the Autoscaling section. How many workers are currently being used to process messages in the Pub/Sub topic?
- Click on "See more history" and review how many workers were used at different points in time during pipeline execution.
- The data from a traffic sensor simulator started at the beginning of the lab creates hundreds of messages per second in the Pub/Sub topic. This will cause Dataflow to increase the number of workers to keep the system lag of the pipeline at optimal levels.
- Click on See more history. In the Worker History pop-up, you can see how Dataflow changed the number of workers. Notice the Rationale column that explains the reason for the change.

#### Task 9: Refresh the sensor data simulation script
- Return to the training_vm SSH terminal where the sensor data is script is running.
- If you see messages that say "INFO: Publishing" then the script is still running. Press CRTL-C to stop it. Then issue the command to start the script again.

```
cd ~/training-data-analyst/courses/streaming/publish
./send_sensor_data.py --speedFactor=60 --project (dollarsign)DEVSHELL_PROJECT_ID
```

- If the script has passed the quota limit, you will see repeating error messages that "credentials could not be refreshed" and you may not be able to use CTRL-C to stop the script. Simply close the SSH terminal. Open a new SSH terminal. The new session will have a fresh quota.
- In the Console, on the Navigation menu () click Compute Engine > VM instances.
- Locate the line with the instance called training_vm.
- On the far right, under 'connect', Click on SSH to open a second terminal window.
- In the training_vm SSH terminal, enter the following to create environment variables.

```
source /training/project_env.sh
cd ~/training-data-analyst/courses/streaming/publish

./send_sensor_data.py --speedFactor=60 --project (dollarsign)DEVSHELL_PROJECT_ID
```

#### Task 10: Stackdriver integration
- Chart Dataflow metrics in Stackdriver Dashboards: Create Dashboards and chart time series of Dataflow metrics.
- Configure Alerts: Define thresholds on job or resource group-level metrics and alert when these metrics reach specified values. Stackdriver alerting can notify on a variety of conditions such as long streaming system lag or failed jobs.
- Monitor User-Defined Metrics: In addition to Dataflow metrics, Dataflow exposes user-defined metrics (SDK Aggregators) as Stackdriver custom counters in the Monitoring UI, available for charting and alerting. Any Aggregator defined in a Dataflow pipeline will be reported to Stackdriver as a custom metric. Dataflow will define a new custom metric on behalf of the user and report incremental updates to Stackdriver approximately every 30 seconds.

#### Task 11: Explore metrics
- Return to the browser tab for Console. On the Navigation menu () click Stackdriver > Monitoring.
- Click Log in with Google.
- Click Create Account.
- Click Continue.
- Click Skip AWS Setup.
- Click Continue.
- Select No Reports and click Continue.
- It may take a few minutes for Stackdriver to import project information about your lab account and the resources already being used. Once the Launch monitoring button becomes active, click Launch monitoring.
- The trial version of Stackdriver provides the Premium Tier of service. So upgrading simply sets up billing so the account will not revert to Basic Tier at the end of 30 days.
- Click on Continue with the trial. (You can also click on 'Dismiss' on the message bar at the top asking if you want to upgrade).
- Explore Stackdriver Metrics
- In the panel to the left click on Resources > Metrics Explorer
- In the Metrics Explorer, find and select the Dataflow_job resource type. You should see a list of available Dataflow-related metrics.


- Select the resource Dataflow Job and the metric Data watermark lag.
- Stackdriver will draw a graph on the right side of the page.
- Under Find resource type and metric, click on the (x) to remove the Data watermark lag metric. Select a new metric, System Lag.
- The metrics that Dataflow provides to Stackdriver are listed here:

https://cloud.google.com/monitoring/api/metrics_gcp

- Data watermark age: The age (time since event timestamp) of the most recent item of data that has been fully processed by the pipeline.
- System lag: The current maximum duration that an item of data has been awaiting processing, in seconds.

#### Task 12: Create alerts
- On the Stackdriver Monitoring click on Stackdriver > Alerting > Policies Overview.
- Click on Add Policy.
- On the Create new Alerting Policy page click on Add Condition.
- On the Metric Threshold row, click Select.
- In the Target section, set the RESOURCE TYPE to Dataflow Job.
- Under APPLIES TO, select Single.
- Select the resource you used in the previous task.
- In the Configuration section, set IF METRIC to System Lag.
- Set CONDITION to above.
- Set THRESHOLD to 5
- Set FOR to 1 minute.
- Click on Save Condition to save the alert.
- Under Notification, click on the pulldown menu to view the options for notification channel. You can set up a notification policy if you would like, using your email address.
- In the Name this policy section, give the policy a name such as MyAlertPolicy.
- Click on Save Policy.
- On the Stackdriver tab, click on Alerting > Events.
- Every time an alert is triggered by a Metric Threshold condition, an Incident and a corresponding Event are created in Stackdriver. If you specified a notification mechanism in the alert (email, SMS, pager, etc), you will also receive a notification.

#### Task 13: Set up dashboards
- On the Stackdriver tab, click on Dashboards > Create dashboard.
- Click on Add Chart.
- On the Add Chart page:
- In the Find resource type and metric box, start typing Dataflow Job and then select it as the Resource Type.
- After you select a Resource Type, the Metric field menu will appear. Select a metric to chart, such as System Lag.
in the Filter panel, select project, then the equals sign '=', then your Project ID.

#### Task 14: Launch another streaming pipeline
- In the training_vm SSH terminal, examine the CurrentConditions.java application. Do not make any changes to the code.
```
cd ~/training-data-analyst/courses/streaming/process/sandiego/src/main/java/com/google/cloud/training/dataanalyst/sandiego 
cat CurrentConditions.java
### basic pipeline
cd ~/training-data-analyst/courses/streaming/process/sandiego
./run_oncloud.sh (dollarsign)DEVSHELL_PROJECT_ID (dollarsign)BUCKET CurrentConditions
```

- Return to the browser tab for Console. On the Navigation menu () click Dataflow and click on the new pipeline job. Confirm that the pipeline job is listed and verify that it is running without errors.
- It will take several minutes before the current_conditions table appears in BigQuery.

## End Lab

## Module 3 Review

1.) The Dataflow models provides constructs that map to the four questions that are relevant in any out-of-order data processing pipeline:

- What results are calculated: Answered via transformations
- Where in event time are results calculated: Answered via Event-time windowing 
- When in processing time are results materialized: Answered via Watermarks, triggers, and allowed lateness.
- How do refinements of results relate: Answered via Accumulation modes


## Lab: Streaming Analytics & Dashboards

#### Objectives:
- Connect to a BigQuery data source
- Create reports and charts to visualize BigQuery data

#### Task 1: Preparation
-In the Console, on the Navigation menu () click Compute Engine > VM instances.
-Locate the line with the instance called training_vm.
-On the far right, under 'connect', Click on SSH to open a terminal window.
-In this lab you will enter CLI commands on the training_vm.
```
ls /training
cp -r /training/training-data-analyst/ .
# set environment variables
source /training/project_env.sh
```

#### Task 2.) Creating a Data Source
- Google Data Studio is a separate service. Open a new browser tab. Navigate to: datastudio.google.com or click on this link: [Google Data Studio](https://datastudio.google.com/)
    - The first step in creating a report in Data Studio is to create a data source for the report. A report may contain one or more data sources. When you create a BigQuery data source, Data Studio uses the BigQuery connector.
    - You must have the appropriate permissions in order to add a BigQuery data source to a Data Studio report. In addition, the permissions applied to BigQuery datasets will apply to the reports, charts, and dashboards you create in Data Studio. When a Data Studio report is shared, the report components are visible only to users who have appropriate permissions.
- On the Reports page, in the Start a new report section, click the Blank template. This starts the account setup process.
- On the Welcome page, click on GET STARTED.
- On the Terms page, click on the checkbox to acknowledge the terms. And click ACCEPT.
- On the Preferences page, select No, thanks for each option to receive email notifications, and click DONE.
- Now that the account is initialized, you need to start the process again.
- On the Reports page, in the Start a new report section, click the Blank template. This time it will take you to a new page and begin an Untitled Report.
- In the Add a data source panel on the right side, click CREATE NEW DATA SOURCE.
- In the Google Connectors column on the left, select BigQuery.
- Click on Authorize.
- In the Sign in dialog, select your Qwiklabs student account.
- Click ALLOW. to give Google Data Studio permission to view the BigQuery resources in your lab account.
- Select My Projects.
- In the Project column, click on your project name.
- In the Dataset column, click on demos.
- In the Table column, click current_conditions.
- In the upper right corner of the window, click CONNECT.
    - Once Data Studio has connected to the BigQuery data source, the table's fields are displayed. You can use this page to adjust the field properties or to create new calculated fields.
- In the upper right corner, click ADD TO REPORT.
- A verification panel opens. Click ADD TO REPORT.
- This will initiate another sign in process to allow Data Studio to access Google Drive.
- In the Sign in dialog, select your Qwiklabs student account.
- Click ALLOW to give Google Data Studio permission to use the Google Drive resources in your lab account.

#### Task 3: Creating a bar chart using a calculated field
- Once you have added the current_conditions data source to the report, the next step is to create a visualization. Begin by creating a bar chart. The bar chart displays the total number of vehicles captured for each highway. To display this, you create a calculated field as follows.
    - (Optional) At the top of the page, click Untitled Report to change the report name. For example, type <PROJECTID>-report1-yourname.
- When the report editor loads, click Insert > Bar chart.
- Using the handle, draw a rectangle on the report to display the chart.
- In the Bar chart properties window, on the Data tab, notice the value for Data Source (current_conditions) and the default values for Dimension and Metric.
- If Dimension is not set to highway, then change Dimension to highway. In the Dimension section, click the existing dimension.
- In the Dimension picker, select highway.
- Click the back arrow to close the Dimension picker.
- In the Metric section, click +Add Metric here and add latitude.
- Click the back arrow.
- In the Metric section, mouse over Record Count and click the (x) to remove it.
- In the Metric section, click the existing metric.
- In the Metric picker, click CREATE NEW METRIC.
- Click the button   Create a calculated field. To display a count of the number of vehicles using each highway, create a calculated field. For this lab, you count the entries in the sensorId field. The value is irrespective, we just need the number of occurrences.
- For Field Name, type vehicles.
- Leave the Field ID unchanged.
- For Formula, type the following (or use the formula assistant): COUNT(sensorId).
- Click SAVE.
- Click DONE.
- In the Metric picker, In the Metric section, click Add metric here.
- Select vehicles. Click the back arrow. (this will display an error)
- Click on the pencil next to Data Source, current_condition.
- Examine the type associated with vehicles. If it is incorrectly set to timestamp, set it to numeric. Click Done. The error is corrected.
- In the Metric section, mouse over latitude and click the (x) to remove it.
- The Dimension should be set to highway and the Metric should be set to vehicles. Notice the chart is sorted in Descending order by default. The highway with the most vehicles are displayed first.
- To enhance the chart, change the bar labels. In the Bar chart properties window, click the STYLE tab.
- In the Bar chart section, check Show data labels.
- The total number of vehicles is displayed above each bar in the chart.
    
#### Task 4: Creating a chart using a custom query
- Because Data Studio does not allow aggregations on metrics, some report components are easier to generate using a custom SQL query. The Custom Query option also lets you leverage BigQuery's full query capabilities such as joins, unions, and analytical functions.
- Alternatively, you can leverage BigQuery's full query capabilities by creating a view. A view is a virtual table defined by a SQL query. You can query data in a view by adding the dataset containing the view as a data source.
- When you specify a SQL query as your BigQuery data source, the results of the query are in table format, which becomes the field definition (schema) for your data source. When you use a custom query as a data source, Data Studio uses your SQL as an inner select statement for each generated query to BigQuery. For more information on custom queries in Data Studio, consult the online help.

- To add a bar chart to your report that uses a custom query data source:
- Click Insert > Bar chart.
- Using the handle, draw a rectangle on the report to display the chart.
- In the Bar chart properties window, on the Data tab, notice the value for Data Source (natality) and the default values for Dimension and Metric are the same as the previous chart. In the Data Source section, click(Select data source).
- Click Create new data source.
- For Google Connectors, click BigQuery.
- For My Projects, click Custom query.
- For Project, select your project.
- Type the following in the Enter custom query window
- At the top of the window, click Untitled data source, and change the data source name to San Diego highway traffic summary.
- In the upper right corner of the window, click Connect. Once Data Studio has connected to the BigQuery data source, the results of the query are used to determine the table schema.
- When the schema is displayed, notice the type and aggregation for each field.
- Click Add to report.
- When prompted, click Add to report.
- In the Bar chart properties, on the Data tab, in the Dimension section, click Invalid metric.
- In the Metric picker, select maxspeed.
- Click the back arrow to close the Metric picker.
- In the Metric section, click +Add metric here.
- In the Metric picker, select minspeed.
- Click the back arrow to close the Metric picker.
- In the Metric section, click Add a metric.
- In the Metric picker, select avgspeed.
- Click the back arrow to close the Metric picker.
- For readability, change the chart styles. In the Bar chart properties, click the Style tab.
- In the Color by section, click on the boxes to select different colors.
- For readability, change the chart styles. In the Bar chart properties, click the Style tab.
- In the Bar chart section, select different colors.

#### Task 5: Viewing your query history
- On the Navigation menu () click BigQuery.
- Refresh the browser window.
- Click Query History.
- The list of queries is displayed with the most recent queries first. Click Open Query to view details on the query such as Job ID and Bytes Processed.

## End Lab

## Module 4 Review

1.) Which of the following is true for Data Studio?
- Data Studio supports data ingest through multiple connectors

2.) Data Studio can issue queries to BigQuery
- True

### BigQuery vs. Big Table
#### BigQuery
- easy, inexpensive
- latency in the order of seconds
- 100k rows / second streaming

#### Bigtable
- low latency/high thoroughput
- 100,000 QPS @ 6ms latency for a 10-node cluster

## Lab: Streaming Data into Big Table
#### Objectives:
- Launch Dataflow pipeline to read from Pub/Sub and write into Bigtable
- Open an HBase shell to query the Bigtable database

#### Task 1: Preparation
- In the Console, on the Navigation menu () click Compute Engine > VM instances.
- Locate the line with the instance called training_vm.
- On the far right, under 'connect', Click on SSH to open a terminal window.
- In this lab you will enter CLI commands on the training_vm.
```
ls /training
cp -r /training/training-data-analyst/ .
# set environment variables
source /training/project_env.sh
# install files
cd ~/training-data-analyst/courses/streaming/process/sandiego
./install_quickstart.sh
```

#### Task 2: Simulate traffic sensor data in Pub / Sub
```
/training/sensor_magic.sh
# start new terminal session for next line
source /training/project_env.sh
cd ~/training-data-analyst/courses/streaming/process/sandiego 
nano run_oncloud.sh
cd ~/training-data-analyst/courses/streaming/process/sandiego
./create_cbt.sh
cd ~/training-data-analyst/courses/streaming/process/sandiego
./run_oncloud.sh $DEVSHELL_PROJECT_ID $BUCKET CurrentConditions --bigtable
cd ~/training-data-analyst/courses/streaming/process/sandiego/quickstart
./quickstart.sh
# hbase(main):001:0>
# indicates it ran successfully
scan 'current_conditions', {'LIMIT' => 2}
scan 'current_conditions', {'LIMIT' => 10, STARTROW => '15#S#1', ENDROW => '15#S#999', COLUMN => 'lane:speed'}
quit
# cleanup
cd ~/training-data-analyst/courses/streaming/process/sandiego
./delete_cbt.sh
```
## End Lab

## Module 5 Review

1.) Which of the following are true about Cloud Bigtable?
- Great for time series data
- Offers low latency in the order of milliseconds
- Support for SQL
- Great for >1 TB of data

Not sure ... 

2.) Cloud Big Table learns access patterns and attempts to distribute reads and storage across nodes evenly.
- True

3.) Which of the following can improve performance of Big Table
- Change schema to minimize data skew
- Clients and Big Table are in the same zone
- Add more nodes